# 1



# 2


In [20]:

# -*- coding: utf-8 -*-
"""
@author: Gemini (Combined Version v3.3 - Stable Fix)
@date: 2025-11-29
@description: 
    整合版不動產交屋稅費計算系統 (v3.3 修正版)
    - 修正 AttributeError: 'Cell' object has no attribute 'wrap_text'
    - Sheet 1: 專業稅費分算計算機 - 最終結算區詳細算式說明
    - Sheet 2: 交屋租金分算表 - 精確日曆天計算
    - Sheet 3: 稅率表
"""
import openpyxl
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.utils import get_column_letter

# --- 共用樣式定義 ---
class Styles:
    fill_yellow_input = PatternFill(start_color='FFFFE0', end_color='FFFFE0', fill_type='solid')
    fill_blue_input = PatternFill(start_color='EAF1FF', end_color='EAF1FF', fill_type='solid')
    fill_title_green = PatternFill(start_color='D7E9D1', end_color='D7E9D1', fill_type='solid')
    fill_header_grey = PatternFill(start_color='E7E6E6', end_color='E7E6E6', fill_type='solid')
    fill_result_orange = PatternFill(start_color='FFF2E6', end_color='FFF2E6', fill_type='solid')
    fill_period_land = PatternFill(start_color='E2EFDA', end_color='E2EFDA', fill_type='solid')
    fill_period_house = PatternFill(start_color='DEEBF7', end_color='DEEBF7', fill_type='solid')
    fill_deadline_red = PatternFill(start_color='FCE4D6', end_color='FCE4D6', fill_type='solid') 
    
    font_title = Font(name='微軟正黑體', size=18, bold=True)
    font_section_header = Font(name='微軟正黑體', size=14, bold=True)
    font_body = Font(name='微軟正黑體', size=12)
    font_body_bold = Font(name='微軟正黑體', size=12, bold=True)
    font_buyer_pays = Font(name='微軟正黑體', size=12, bold=True, color="0000FF")
    font_seller_pays = Font(name='微軟正黑體', size=12, bold=True, color="C00000")
    font_red_bold = Font(name='微軟正黑體', size=12, bold=True, color="C00000")
    font_final_result = Font(name='微軟正黑體', size=16, bold=True, color="C00000")
    font_note = Font(name='微軟正黑體', size=11, italic=True)
    font_footer = Font(name='微軟正黑體', size=10, color="808080")
    
    align_center = Alignment(horizontal='center', vertical='center', wrap_text=True)
    align_left = Alignment(horizontal='left', vertical='center', wrap_text=True)
    align_right = Alignment(horizontal='right', vertical='center', wrap_text=True)
    # 此處已包含 wrap_text=True
    align_left_top = Alignment(horizontal='left', vertical='top', wrap_text=True)
    
    thin_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    medium_border = Border(left=Side(style='medium'), right=Side(style='medium'), top=Side(style='medium'), bottom=Side(style='medium'))

def apply_styles_to_range(ws, cell_range, styles_dict):
    """輔助函式：對指定的儲存格範圍一次性套用多種樣式"""
    rows = ws[cell_range]
    if not isinstance(rows, tuple):
        rows = ((rows,),)
    for row in rows:
        for cell in row:
            for style_name, style_obj in styles_dict.items():
                if style_obj is not None:
                    setattr(cell, style_name, style_obj)

# --- 租金分算表邏輯 (v7 精確版) ---
def generate_rent_sheet(wb):
    ws = wb.create_sheet('交屋租金分算表')
    ws.sheet_view.showGridLines = False
    
    ws.page_setup.orientation = ws.ORIENTATION_LANDSCAPE
    ws.page_setup.paperSize = ws.PAPERSIZE_A4
    ws.page_setup.fitToWidth = 1; ws.page_setup.fitToHeight = 0

    # 樣式定義
    COLOR_HEADER_BG = "1E293B"; COLOR_INPUT_BG = "FFFFFF"
    COLOR_SELLER_BG = "FFF1F2"; COLOR_SELLER_HEADER = "E11D48"; COLOR_SELLER_TXT = "881337"
    COLOR_BUYER_BG = "ECFDF5"; COLOR_BUYER_HEADER = "059669"; COLOR_BUYER_TXT = "064E3B"
    COLOR_FINAL_BG = "FFEDD5"; COLOR_BORDER = "CBD5E1"; COLOR_BORDER_THICK = "334155"
    COLOR_SIG_BG = "F8FAFC"; COLOR_SUBTOTAL_BG = "F1F5F9"

    font_header_main = Font(name='微軟正黑體', size=18, bold=True, color='1E293B')
    font_header_base = Font(name='微軟正黑體', size=11, bold=True, color='FFFFFF')
    font_normal = Font(name='微軟正黑體', size=11, color='334155')
    font_input = Font(name='微軟正黑體', size=11, color='0F172A', bold=True)
    font_hint = Font(name='微軟正黑體', size=9, color='64748B')
    font_seller = Font(name='微軟正黑體', size=10, color=COLOR_SELLER_TXT, bold=True)
    font_buyer = Font(name='微軟正黑體', size=10, color=COLOR_BUYER_TXT, bold=True)
    font_subtotal = Font(name='微軟正黑體', size=11, bold=True, color='1E293B')
    font_big_res = Font(name='微軟正黑體', size=14, color='9A3412', bold=True)
    font_sig_title = Font(name='微軟正黑體', size=12, bold=True, color='000000')

    align_c = Alignment(horizontal='center', vertical='center', wrap_text=True)
    align_l = Alignment(horizontal='left', vertical='center', wrap_text=True)
    align_r = Alignment(horizontal='right', vertical='center', wrap_text=True)

    border_std = Border(left=Side(style='thin', color=COLOR_BORDER), right=Side(style='thin', color=COLOR_BORDER), top=Side(style='thin', color=COLOR_BORDER), bottom=Side(style='thin', color=COLOR_BORDER))
    border_thick_right = Border(left=Side(style='thin', color=COLOR_BORDER), right=Side(style='medium', color=COLOR_BORDER_THICK), top=Side(style='thin', color=COLOR_BORDER), bottom=Side(style='thin', color=COLOR_BORDER))
    border_thick_side = Side(style='thick', color='334155')

    def set_outer_border(ws, row_start, row_end, col_start, col_end):
        for c in range(col_start, col_end + 1):
            ws.cell(row=row_start, column=c).border = Border(left=ws.cell(row=row_start, column=c).border.left, right=ws.cell(row=row_start, column=c).border.right, top=border_thick_side, bottom=ws.cell(row=row_start, column=c).border.bottom)
            ws.cell(row=row_end, column=c).border = Border(left=ws.cell(row=row_end, column=c).border.left, right=ws.cell(row=row_end, column=c).border.right, top=ws.cell(row=row_end, column=c).border.top, bottom=border_thick_side)
        for r in range(row_start, row_end + 1):
            ws.cell(row=r, column=col_start).border = Border(left=border_thick_side, right=ws.cell(row=r, column=col_start).border.right, top=ws.cell(row=r, column=col_start).border.top, bottom=ws.cell(row=r, column=col_start).border.bottom)
            ws.cell(row=r, column=col_end).border = Border(left=ws.cell(row=r, column=col_end).border.left, right=border_thick_side, top=ws.cell(row=r, column=col_end).border.top, bottom=ws.cell(row=r, column=col_end).border.bottom)

    def set_cell(coord, value, font=font_normal, fill=None, align=align_c, border=border_std, fmt=None):
        c = ws[coord]; c.value = value; c.font = font; c.alignment = align
        if border: c.border = border
        if fill: c.fill = PatternFill('solid', fgColor=fill)
        if fmt: c.number_format = fmt

    # Setup Columns
    ws.column_dimensions['A'].width = 15; ws.column_dimensions['B'].width = 11; ws.column_dimensions['C'].width = 11
    ws.column_dimensions['D'].width = 12; ws.column_dimensions['E'].width = 12; ws.column_dimensions['F'].width = 11
    ws.column_dimensions['G'].width = 30; ws.column_dimensions['H'].width = 12; ws.column_dimensions['I'].width = 30; ws.column_dimensions['J'].width = 12

    # Header
    set_cell("A1", "交屋租金分算表 (明細)", font=font_header_main, align=align_l, border=None)
    ws.merge_cells("A1:J1"); ws.row_dimensions[1].height = 35

    # Basic Info
    set_cell("A2", "買方:", font=font_header_base, fill=COLOR_HEADER_BG, align=align_r)
    set_cell("B2", "（請填寫）", font=font_input, fill=COLOR_INPUT_BG, align=align_l)
    ws.merge_cells("B2:E2")
    set_cell("F2", "賣方:", font=font_header_base, fill=COLOR_HEADER_BG, align=align_r)
    set_cell("G2", "（請填寫）", font=font_input, fill=COLOR_INPUT_BG, align=align_l)
    ws.merge_cells("G2:J2"); ws.row_dimensions[2].height = 25

    set_cell("A3", "房屋地址:", font=font_header_base, fill=COLOR_HEADER_BG, align=align_r)
    set_cell("B3", "（請填寫）", font=font_input, fill=COLOR_INPUT_BG, align=align_l)
    ws.merge_cells("B3:J3"); ws.row_dimensions[3].height = 25

    set_cell("A4", "交屋日期 (7碼)", font=font_header_base, fill=COLOR_HEADER_BG)
    set_cell("B4", 1141127, font=font_input, fill=COLOR_INPUT_BG, fmt="0")
    set_cell("C4", f'=IF(B4="","", TEXT(DATE(1911+LEFT(B4,3),MID(B4,4,2),RIGHT(B4,2)),"yyyy/mm/dd"))', font=font_input, align=align_l, border=None)
    set_cell("D4", "(此日期將連動計算下方租金拆分)", font=Font(name='微軟正黑體', size=10, color='E11D48', bold=True), align=align_l, border=None)
    ws.merge_cells("D4:J4"); ws.row_dimensions[4].height = 25

    set_cell("A5", "交屋日算誰的", font=font_header_base, fill=COLOR_HEADER_BG)
    set_cell("B5", "歸賣方", font=font_input, fill=COLOR_INPUT_BG)
    ws.merge_cells("C5:E5")
    set_cell("C5", '="⚖️ 權益切割：交屋日當天歸 " & RIGHT(B5,2)', font=font_hint, align=align_l, border=None)
    dv_owner = DataValidation(type='list', formula1='"歸買方,歸賣方"', allow_blank=False); ws.add_data_validation(dv_owner); dv_owner.add('B5:B5')
    ws.row_dimensions[5].height = 25

    set_cell("A6", "計算基準", font=font_header_base, fill=COLOR_HEADER_BG)
    set_cell("B6", "精確分段計算", font=font_normal, fill="F1F5F9")
    ws.merge_cells("C6:F6")
    set_cell("C6", '="📊 規則：依該月實際天數計算 (例：9月除以30、10月除以31)，精確反映日租金權重。"', font=font_hint, align=align_l, border=None)
    ws.row_dimensions[6].height = 25

    # Data Table Headers
    headers = ['房號/租客', '租期起', '租期訖', '月租金', '押金', '租客已繳至', '賣方補貼買方\n(計算詳情)', '補貼金額', '買方補貼賣方\n(計算詳情)', '補貼金額']
    ws.row_dimensions[7].height = 40
    for i, h in enumerate(headers):
        col = get_column_letter(i+1)
        bg_color = '64748B'
        if '賣方' in h: bg_color = COLOR_SELLER_HEADER
        if '買方' in h: bg_color = COLOR_BUYER_HEADER
        cell_border = border_std
        if col == 'F': cell_border = border_thick_right
        set_cell(f'{col}7', h, font=font_header_base, fill=bg_color, border=cell_border)

    # Core Logic Formulas
    def get_prorated_rent_value_formula(rent_ref, s_date, e_date):
        cond_same = f"(EOMONTH({s_date},0)=EOMONTH({e_date},0))"
        factor_same = f"(({e_date}-{s_date}+1)/DAY(EOMONTH({s_date},0)))"
        p_start = f"((EOMONTH({s_date},0)-{s_date}+1)/DAY(EOMONTH({s_date},0)))"
        p_end = f"(DAY({e_date})/DAY(EOMONTH({e_date},0)))"
        p_mid = f"MAX(0, (YEAR({e_date})-YEAR({s_date}))*12 + MONTH({e_date})-MONTH({s_date}) - 1)"
        factor_cross = f"({p_start} + {p_end} + {p_mid})"
        return f"ROUND({rent_ref} * IF({cond_same}, {factor_same}, {factor_cross}), 0)"

    def get_prorated_rent_text_formula(rent_ref, s_date, e_date):
        days_same = f"({e_date}-{s_date}+1)"; total_same = f"DAY(EOMONTH({s_date},0))"
        txt_same_math = f""" "$" & {rent_ref} & "×(" & {days_same} & "/" & {total_same} & ")" """
        txt_same = f""" TEXT({s_date}, "mm/dd") & "~" & TEXT({e_date}, "mm/dd") & " (" & {days_same} & "天)" & CHAR(10) & {txt_same_math} """
        d1 = f"(EOMONTH({s_date},0)-{s_date}+1)"; t1 = f"DAY(EOMONTH({s_date},0))"
        d2 = f"DAY({e_date})"; t2 = f"DAY(EOMONTH({e_date},0))"
        txt_cross_math = f""" "$" & {rent_ref} & "×(" & {d1} & "/" & {t1} & ") + $" & {rent_ref} & "×(" & {d2} & "/" & {t2} & ")" """
        is_multi = f"((YEAR({e_date})-YEAR({s_date}))*12 + MONTH({e_date})-MONTH({s_date}) > 1)"
        txt_cross_math_final = f""" IF({is_multi}, {txt_cross_math} & " +...", {txt_cross_math}) """
        days_total = f"({e_date}-{s_date}+1)"
        txt_cross = f""" TEXT({s_date}, "mm/dd") & "~" & TEXT({e_date}, "mm/dd") & " (" & {days_total} & "天)" & CHAR(10) & {txt_cross_math_final} """
        cond_same = f"(EOMONTH({s_date},0)=EOMONTH({e_date},0))"
        return f"IF({cond_same}, {txt_same}, {txt_cross})"

    start_row = 8; end_row = 19
    for r in range(start_row, end_row + 1):
        row_str = str(r); ws.row_dimensions[r].height = 35
        for c_char in ['A', 'B', 'C', 'D', 'E']: set_cell(f'{c_char}{row_str}', '', fill=COLOR_INPUT_BG)
        set_cell(f'F{row_str}', '', fill=COLOR_INPUT_BG, border=border_thick_right)
        
        ws[f'B{row_str}'].number_format = '0'; ws[f'C{row_str}'].number_format = '0'
        ws[f'D{row_str}'].number_format = '#,##0'; ws[f'E{row_str}'].number_format = '#,##0'; ws[f'F{row_str}'].number_format = '0'

        safe_b4 = f"IF($B$4=\"\", 1140101, $B$4)"
        d_handover = f"DATE(1911+LEFT({safe_b4},3),MID({safe_b4},4,2),RIGHT({safe_b4},2))"
        safe_f = f"IF(F{row_str}=\"\", {safe_b4}, F{row_str})"
        d_paid = f"DATE(1911+LEFT({safe_f},3),MID({safe_f},4,2),RIGHT({safe_f},2))"
        d_cut = f"IF($B$5=\"歸買方\", {d_handover}, {d_handover}+1)"
        is_valid = f'AND(D{row_str}<>"", F{row_str}<>"")'
        rent_ref = f"D{row_str}"

        # Logic 1: Seller pays Buyer
        cond_seller = f"({d_paid} >= {d_cut})"
        s_val = get_prorated_rent_value_formula(rent_ref, d_cut, d_paid)
        s_txt = get_prorated_rent_text_formula(rent_ref, d_cut, d_paid)
        set_cell(f"G{row_str}", f"""=IF({is_valid}, IF({cond_seller}, {s_txt}, ""), "")""", fill=COLOR_SELLER_BG, font=font_seller)
        ws[f"G{row_str}"].alignment = Alignment(wrap_text=True, horizontal="center", vertical="center")
        set_cell(f"H{row_str}", f"""=IF({is_valid}, IF({cond_seller}, {s_val}, 0), "")""", fill=COLOR_SELLER_BG, fmt="#,##0", font=font_seller)

        # Logic 2: Buyer pays Seller
        b_date_start = f"({d_paid}+1)"; b_date_end = f"({d_cut}-1)"; cond_buyer = f"({d_cut} > {d_paid} + 1)"
        b_val = get_prorated_rent_value_formula(rent_ref, b_date_start, b_date_end)
        b_txt = get_prorated_rent_text_formula(rent_ref, b_date_start, b_date_end)
        set_cell(f"I{row_str}", f"""=IF({is_valid}, IF({cond_buyer}, {b_txt}, ""), "")""", fill=COLOR_BUYER_BG, font=font_buyer)
        ws[f"I{row_str}"].alignment = Alignment(wrap_text=True, horizontal="center", vertical="center")
        set_cell(f"J{row_str}", f"""=IF({is_valid}, IF({cond_buyer}, {b_val}, 0), "")""", fill=COLOR_BUYER_BG, fmt="#,##0", font=font_buyer)

    # Example Data
    set_cell("A8", "301 (範例)", fill=COLOR_INPUT_BG); set_cell("B8", 1140915, fill=COLOR_INPUT_BG, fmt="0")
    set_cell("C8", 1141014, fill=COLOR_INPUT_BG, fmt="0"); set_cell("D8", 28300, fill=COLOR_INPUT_BG, fmt="#,##0")
    set_cell("E8", 52000, fill=COLOR_INPUT_BG, fmt="#,##0"); set_cell("F8", 1140914, fill=COLOR_INPUT_BG, fmt="0", border=border_thick_right) 
    
    set_outer_border(ws, 7, 19, 1, 10)

    # Totals Section
    sum_start_row = end_row + 2
    ws.merge_cells(f'A{sum_start_row}:D{sum_start_row}'); set_cell(f'A{sum_start_row}', '賣方應付項目 (Debits)', font=font_header_base, fill=COLOR_SELLER_HEADER, align=align_l)
    ws.merge_cells(f'F{sum_start_row}:J{sum_start_row}'); set_cell(f'F{sum_start_row}', '買方應付項目及費用 (Credits)', font=font_header_base, fill=COLOR_BUYER_HEADER, align=align_l)
    ws.row_dimensions[sum_start_row].height = 30

    seller_items = [
        ('1. 押金移轉', f'=SUM(E{start_row}:E{end_row})', False), 
        ('2. 賣方補貼買方 (預收租金)', f'=SUM(H{start_row}:H{end_row})', False), 
        ('3. 其他費用 (1)', 0, True), ('4. 其他費用 (2)', 0, True), ('5. 其他費用 (3)', 0, True), ('6. 其他費用 (4)', 0, True)
    ]
    buyer_items = [
        ('1. 買方補貼賣方 (欠繳租金)', f'=SUM(J{start_row}:J{end_row})', False), 
        ('2. 機上盒/網路/水電等雜支', 0, True), 
        ('3. 其他費用 (1)', 0, True), ('4. 其他費用 (2)', 0, True), ('5. 其他費用 (3)', 0, True), ('6. 其他費用 (4)', 0, True)
    ]
    
    max_len = 6
    seller_amount_cells = []; buyer_amount_cells = []; start_item_row = sum_start_row + 1
    for i in range(max_len):
        r = start_item_row + i; ws.row_dimensions[r].height = 25
        ws.merge_cells(f'A{r}:C{r}')
        name, val, is_input = seller_items[i]
        set_cell(f'A{r}', name, align=align_l, border=None, fill=COLOR_INPUT_BG if "其他" in name else None)
        set_cell(f'D{r}', val, fmt='#,##0', font=font_seller, align=align_r, fill=COLOR_INPUT_BG if is_input else None)
        seller_amount_cells.append(f'D{r}')
        ws.merge_cells(f'F{r}:I{r}')
        name, val, is_input = buyer_items[i]
        set_cell(f'F{r}', name, align=align_l, border=None, fill=COLOR_INPUT_BG if "其他" in name else None)
        set_cell(f'J{r}', val, fmt='#,##0', font=font_buyer, align=align_r, fill=COLOR_INPUT_BG if is_input else None)
        buyer_amount_cells.append(f'J{r}')

    subtotal_row = start_item_row + max_len
    ws.row_dimensions[subtotal_row].height = 30
    ws.merge_cells(f'A{subtotal_row}:C{subtotal_row}'); set_cell(f'A{subtotal_row}', '賣方應付小計', font=font_subtotal, fill=COLOR_SUBTOTAL_BG, align=align_r)
    set_cell(f'D{subtotal_row}', f'={"+".join(seller_amount_cells)}', fmt='#,##0', font=font_subtotal, fill=COLOR_SUBTOTAL_BG, align=align_r)
    ws.merge_cells(f'F{subtotal_row}:I{subtotal_row}'); set_cell(f'F{subtotal_row}', '買方應付小計', font=font_subtotal, fill=COLOR_SUBTOTAL_BG, align=align_r)
    set_cell(f'J{subtotal_row}', f'={"+".join(buyer_amount_cells)}', fmt='#,##0', font=font_subtotal, fill=COLOR_SUBTOTAL_BG, align=align_r)

    # Borders
    set_outer_border(ws, sum_start_row, subtotal_row, 1, 4)
    set_outer_border(ws, sum_start_row, subtotal_row, 6, 10)

    # Final Result
    final_row = subtotal_row + 1; ws.merge_cells(f'A{final_row}:J{final_row}'); ws.row_dimensions[final_row].height = 50
    ws[f'K{final_row}'] = f'=(D{subtotal_row}) - (J{subtotal_row})' 
    text_formula = f"""=IF(K{final_row}>0, "最終結算：賣方應找補買方 $" & TEXT(K{final_row}, "#,##0"), IF(K{final_row}<0, "最終結算：買方應找補賣方 $" & TEXT(ABS(K{final_row}), "#,##0"), "最終結算：雙方無須找補"))"""
    set_cell(f'A{final_row}', text_formula, font=font_big_res, align=align_c, fill=COLOR_FINAL_BG)
    set_outer_border(ws, final_row, final_row, 1, 10)

    # Signatures & Remarks
    sig_row = final_row + 2; ws.row_dimensions[sig_row].height = 60
    ws.merge_cells(f'A{sig_row}:D{sig_row}'); set_cell(f'A{sig_row}', "買方簽章：\n\n__________________________________", font=font_sig_title, align=Alignment(horizontal='left', vertical='top', wrap_text=True))
    set_outer_border(ws, sig_row, sig_row, 1, 4)
    ws.merge_cells(f'F{sig_row}:I{sig_row}'); set_cell(f'F{sig_row}', "賣方簽章：\n\n__________________________________", font=font_sig_title, align=Alignment(horizontal='left', vertical='top', wrap_text=True))
    set_outer_border(ws, sig_row, sig_row, 6, 9)

    remark_row = sig_row + 2
    ws.merge_cells(f'A{remark_row}:J{remark_row+2}')
    for r in range(remark_row, remark_row + 3): ws.row_dimensions[r].height = 25
    set_cell(f'A{remark_row}', "備註事項：\n", font=font_normal, align=Alignment(horizontal='left', vertical='top', wrap_text=True))
    set_outer_border(ws, remark_row, remark_row+2, 1, 10)

    return (f"'交屋租金分算表'!D{subtotal_row}", f"'交屋租金分算表'!J{subtotal_row}")

# --- 主表邏輯 (v3.3 修正版) ---
def generate_tax_sheet(wb, rent_subtotals_refs):
    ws = wb.create_sheet("專業稅費分算計算機", 0)
    
    # 欄寬與列高
    for col, width in {'A': 20, 'B': 18, 'C': 18, 'D': 18, 'E': 15, 'F': 22, 'G':18, 'H': 55}.items():
        ws.column_dimensions[col].width = width
    
    for i in range(1, 50): ws.row_dimensions[i].height = 32
    
    ws.row_dimensions[1].height = 50 
    ws.row_dimensions[11].hidden = True
    ws.row_dimensions[19].height = 180
    ws.row_dimensions[20].height = 180
    ws.row_dimensions[39].height = 55 

    ws.column_dimensions['AA'].hidden = True; ws.column_dimensions['AB'].hidden = True

    # 基礎資料與日期解析
    ws['AA1'] = '=IFERROR(IF(LEN(C2)=7, DATE(VALUE(LEFT(C2,3))+1911, VALUE(MID(C2,4,2)), VALUE(RIGHT(C2,2))), IF(LEN(C2)=6, DATE(VALUE(LEFT(C2,2))+1911, VALUE(MID(C2,3,2)), VALUE(RIGHT(C2,2))), "")), "")'
    ws['AA2'] = '=IFERROR(IF(LEN(C3)=7, DATE(VALUE(LEFT(C3,3))+1911, VALUE(MID(C3,4,2)), VALUE(RIGHT(C3,2))), IF(LEN(C3)=6, DATE(VALUE(LEFT(C3,2))+1911, VALUE(MID(C3,3,2)), VALUE(RIGHT(C3,2))), "")), "")'
    ws['AA4'] = '=IF(C4="是", 1, 0)'
    ws['AA5'] = '=IF(AA2="","",AND(DAY(EOMONTH(AA2,0))=DAY(AA2), C4="否"))'
    
    for i in range(7, 10):
        ws[f'AA{i}'] = f'=IFERROR(IF(ISNUMBER(FIND("/",E{i})), LEFT(E{i}, FIND("/", E{i})-1) / MID(E{i}, FIND("/", E{i})+1, LEN(E{i})), IF(ISNUMBER(FIND("%",E{i})), VALUE(SUBSTITUTE(E{i},"%",""))/100, IF(ISNUMBER(E{i}), E{i}/100, 1))), 1)'

    # 稅額計算邏輯
    ws['AA11'] = '=ROUND(SUMPRODUCT((F7:F9="自用住宅 (2‰)")*G7:G9*0.002),0)'
    ws['AA12'] = '=SUMPRODUCT((F7:F9<>"自用住宅 (2‰)")*G7:G9)'
    ws['AA13'] = '=IF(C10="",0,C10)'
    ws['AA14'] = '=ROUND(IF(AA13=0, AA12*0.01, IF(AA12<=AA13, AA12*0.01, IF(AA12<=AA13*5, AA12*0.015-AA13*0.005, IF(AA12<=AA13*10, AA12*0.025-AA13*0.065, IF(AA12<=AA13*15, AA12*0.035-AA13*0.175, IF(AA12<=AA13*20, AA12*0.045-AA13*0.335, AA12*0.055-AA13*0.545)))))),0)'
    ws['AA15'] = '=AA11+AA14'
    ws['AA16'] = '=IFERROR(IF(ISNUMBER(FIND("-",C12)), VALUE(LEFT(C12, FIND("-",C12)-1)), C12), C12)'
    ws['AA17'] = '=IFERROR(IF(ISNUMBER(FIND("-",C12)), VALUE(MID(C12, FIND("-",C12)+1, LEN(C12))), 0), 0)'
    ws['AA18'] = '=IF(ISNUMBER(C12), C12, AA16-AA17)'
    
    # 雜項費用日期
    ws['AA20'] = '=IFERROR(IF(LEN(F22)=7, DATE(VALUE(LEFT(F22,3))+1911, VALUE(MID(F22,4,2)), VALUE(RIGHT(F22,2))), IF(LEN(F22)=6, DATE(VALUE(LEFT(F22,2))+1911, VALUE(MID(F22,3,2)), VALUE(RIGHT(F22,2))), "")), "")'
    ws['AA21'] = '=IFERROR(IF(LEN(F23)=7, DATE(VALUE(LEFT(F23,3))+1911, VALUE(MID(F23,4,2)), VALUE(RIGHT(F23,2))), IF(LEN(F23)=6, DATE(VALUE(LEFT(F23,2))+1911, VALUE(MID(F23,3,2)), VALUE(RIGHT(F23,2))), "")), "")'

    # 核心公式 (AB區)
    calc_formulas = {
        'AB1': '=IF(C15<>"", C15, AA15)',
        'AB2': '=IF(C16<>"", C16, IFERROR(ROUND(AA18*VLOOKUP(C13, 稅率表!$A$2:$B$15, 2, FALSE),0), ""))',
        'AB3': '=IF(AA5, 12, IFERROR(DATE(YEAR(AA2),12,31)-DATE(YEAR(AA2),1,1)+1, ""))',
        'AB4': '=IFERROR(DATE(YEAR(AA2)-IF(MONTH(AA2)<7,1,0),7,1), "")',
        'AB5': '=IFERROR(DATE(YEAR(AA2)-IF(MONTH(AA2)<7,1,0)+1,6,30), "")',
        'AB6': '=IF(AA5, 12, IFERROR(AB5-AB4+1, ""))',
        'AB7': '=IFERROR(EOMONTH(DATE(YEAR(AB5),2,1),0), "")',
        'AB8': '=IFERROR(IF(AA1>DATE(YEAR(AA2),8,31), "賣方", "買方"), "")',
        'AB9': '=IFERROR(IF(AA1>AB7, "賣方", "買方"), "")',
        'AB10': '=IFERROR(IF(AA5, MONTH(AA2), AA2-DATE(YEAR(AA2),1,1)+1-AA4),"")',
        'AB11': '=IFERROR(IF(AA5, 12-MONTH(AA2), DATE(YEAR(AA2),12,31)-AA2+AA4),"")',
        'AB12': '=IFERROR(IF(AA5, IF(MONTH(AA2)>=7, MONTH(AA2)-6, MONTH(AA2)+6), AA2-AB4+1-AA4),"")',
        'AB13': '=IFERROR(IF(AA5, 12 - IF(MONTH(AA2)>=7, MONTH(AA2)-6, MONTH(AA2)+6), AB5-AA2+AA4),"")',
        'AB21': '=IFERROR(IF(AND(B22>0, AA20>=AA2), IF((YEAR(AA20)=YEAR(AA2))*(MONTH(AA20)=MONTH(AA2)), ROUND(B22*(AA20-AA2+AA4)/DAY(EOMONTH(AA2,0)),0), ROUND((B22*(DAY(EOMONTH(AA2,0))-DAY(AA2)+AA4)/DAY(EOMONTH(AA2,0))) + (B22*MAX(0,(YEAR(AA20)-YEAR(AA2))*12+MONTH(AA20)-MONTH(AA2)-1)) + (B22*DAY(AA20)/DAY(EOMONTH(AA20,0))),0)), 0), 0)',
        'AB22': '=IFERROR(IF(AND(B23>0, AA21>=AA2), IF((YEAR(AA21)=YEAR(AA2))*(MONTH(AA21)=MONTH(AA2)), ROUND(B23*(AA21-AA2+AA4)/DAY(EOMONTH(AA2,0)),0), ROUND((B23*(DAY(EOMONTH(AA2,0))-DAY(AA2)+AA4)/DAY(EOMONTH(AA2,0))) + (B23*MAX(0,(YEAR(AA21)-YEAR(AA2))*12+MONTH(AA21)-MONTH(AA2)-1)) + (B23*DAY(AA21)/DAY(EOMONTH(AA21,0))),0)), 0), 0)',
        'AB24': '=IFERROR(IF(AND(B22>0, AA20<AA2), ROUND((B22*(DAY(EOMONTH(AA20,0))-DAY(AA20))/DAY(EOMONTH(AA20,0))) + (B22*MAX(0,(YEAR(AA2)-YEAR(AA20))*12+MONTH(AA2)-MONTH(AA20)-1)) + (B22*DAY(AA2)/DAY(EOMONTH(AA2,0))),0), 0), 0)',
        'AB25': '=IFERROR(IF(AND(B23>0, AA21<AA2), ROUND((B23*(DAY(EOMONTH(AA21,0))-DAY(AA21))/DAY(EOMONTH(AA21,0))) + (B23*MAX(0,(YEAR(AA2)-YEAR(AA21))*12+MONTH(AA2)-MONTH(AA21)-1)) + (B23*DAY(AA2)/DAY(EOMONTH(AA2,0))),0), 0), 0)',
    }
    for k, v in calc_formulas.items(): ws[k] = v

    # --- UI Layout ---
    ws.merge_cells('A1:H1'); ws['A1'] = "不動產買賣稅費分算計算機 v5.33 (租金獨立版)"
    apply_styles_to_range(ws, 'A1', {'font': Styles.font_title, 'fill': Styles.fill_title_green, 'alignment': Styles.align_center})
    
    # Dates Row 2
    ws['A2'] = "權狀登記(過戶)日期"; ws.merge_cells('A2:B2')
    ws['C2'] = 1141008; 
    ws.merge_cells('D2:E2'); ws['D2'] = '=IF(AA1<>"", TEXT(AA1, "民國 e 年 m 月 d 日"), "請依左側格式輸入日期")'
    ws['F2'] = "房地合一稅申報期限"
    ws.merge_cells('G2:H2')
    ws['G2'] = '=IF(AA1<>"", TEXT(AA1+30, "民國 e 年 m 月 d 日"), "")' 
    apply_styles_to_range(ws, 'F2', {'font': Styles.font_body_bold, 'fill': Styles.fill_deadline_red, 'alignment': Styles.align_center})
    apply_styles_to_range(ws, 'G2:H2', {'font': Styles.font_red_bold, 'fill': Styles.fill_deadline_red, 'alignment': Styles.align_left})

    # Dates Row 3
    ws['A3'] = "房屋點交(交屋)日期"; ws.merge_cells('A3:B3') 
    ws['C3'] = 1141008; 
    ws.merge_cells('D3:E3'); ws['D3'] = '=IF(AA2<>"", TEXT(AA2, "民國 e 年 m 月 d 日"), "請依左側格式輸入日期")'
    ws.merge_cells('F3:H4')
    ws['F3'] = '=IF(AA1="","",IF(MOD(YEAR(AA1)-1911,2)=1, "【注意】\n民國 " & YEAR(AA1)-1911 & " 年為地價調整年！\n前一年稅單僅供參考。", "【資訊】\n民國 " & YEAR(AA1)-1911 & " 年非地價調整年。"))'
    apply_styles_to_range(ws, 'F3', {'font': Styles.font_red_bold, 'alignment': Styles.align_center})

    # Dates Row 4
    ws['A4'] = "是否帶租約交屋"; ws.merge_cells('A4:B4')
    ws['C4'] = "否"
    ws.merge_cells('D4:E4'); ws['D4'] = '=IF(AA5, "【按月分算模式】", "【按日分算模式】") & " 交屋日歸屬：" & IF(C4="是", "買方", "賣方")'
    
    apply_styles_to_range(ws, 'A2:B4', {'font': Styles.font_body, 'fill': Styles.fill_header_grey, 'alignment': Styles.align_right})
    apply_styles_to_range(ws, 'C2:C4', {'font': Styles.font_body, 'fill': Styles.fill_yellow_input, 'alignment': Styles.align_center, 'number_format': '@'})
    apply_styles_to_range(ws, 'D2:E4', {'font': Styles.font_body, 'alignment': Styles.align_left})
    apply_styles_to_range(ws, 'D4', {'font': Styles.font_body_bold})

    dv_lease = DataValidation(type="list", formula1='"是,否"'); dv_lease.add('C4'); ws.add_data_validation(dv_lease)

    # Section 1
    ws.merge_cells('A5:H5'); ws['A5'] = "（一）稅額計算模式"
    apply_styles_to_range(ws, 'A5', {'font': Styles.font_section_header, 'fill': Styles.fill_header_grey, 'alignment': Styles.align_center})
    ws['A6'] = "模式一：多筆地價詳算"; apply_styles_to_range(ws, 'A6', {'font': Styles.font_body_bold, 'fill': Styles.fill_header_grey, 'alignment': Styles.align_center})
    for i, h in enumerate(["地號/說明", "申報地價", "面積(m²)", "權利範圍", "土地類型", "地價總額"], 2): ws.cell(row=6, column=i, value=h)
    apply_styles_to_range(ws, 'B6:G6', {'font': Styles.font_body_bold, 'alignment': Styles.align_center})
    
    for i in range(7, 10):
        ws[f'A{i}'] = f"土地 {i-6}"; ws[f'G{i}'] = f'=IFERROR(C{i}*D{i}*AA{i}, "")'
    ws['B7']="甲地"; ws['C7']=10000; ws['D7']=70; ws['E7']="100%"; ws['F7']="一般用地 (10‰)"
    apply_styles_to_range(ws, 'A7:A9', {'font': Styles.font_body, 'alignment': Styles.align_right})
    apply_styles_to_range(ws, 'B7:F9', {'fill': Styles.fill_yellow_input, 'alignment': Styles.align_center})
    apply_styles_to_range(ws, 'E7:E9', {'number_format': '@'})
    apply_styles_to_range(ws, 'G7:G9', {'number_format': '#,##0', 'alignment': Styles.align_center})
    
    ws['A10'] = "累進起點地價"; ws.merge_cells('A10:B10'); ws['C10'] = 1700000
    apply_styles_to_range(ws, 'C10', {'fill': Styles.fill_yellow_input, 'alignment': Styles.align_center, 'number_format': '#,##0'})
    ws.merge_cells('D10:H10'); ws['D10'] = "請輸入該縣市累進起點地價，留空則全按基本稅率(10‰)計算。"
    apply_styles_to_range(ws, 'D10', {'font': Styles.font_note, 'alignment': Styles.align_left})
    
    ws['A12'] = "房屋現值"; ws.merge_cells('A12:B12'); ws['C12'] = "1000000"
    ws['A13'] = "房屋使用情境"; ws.merge_cells('A13:B13'); ws['C13'] = "自住用-全國3戶內 (1.2%)"
    ws.merge_cells('D13:E13'); ws['D13'] = '=IFERROR(VLOOKUP(C13, 稅率表!$A$2:$B$15, 2, FALSE), "")'; ws['D13'].number_format = '0.00%'
    apply_styles_to_range(ws, 'C12:C13', {'fill': Styles.fill_yellow_input, 'alignment': Styles.align_center})
    
    ws.merge_cells('A14:H14'); ws['A14'] = "模式二：快速稅額輸入 (若此區有值，將優先採用)"
    apply_styles_to_range(ws, 'A14', {'font': Styles.font_body_bold, 'fill': Styles.fill_header_grey, 'alignment': Styles.align_center})
    ws.merge_cells('A15:B15'); ws['A15'] = "年度應納地價稅"; apply_styles_to_range(ws, 'C15', {'fill': Styles.fill_blue_input, 'number_format': '#,##0'})
    ws.merge_cells('A16:B16'); ws['A16'] = "年度應納房屋稅"; apply_styles_to_range(ws, 'C16', {'fill': Styles.fill_blue_input, 'number_format': '#,##0'})

    # Section 2
    ws.merge_cells('A17:H17'); ws['A17'] = "（二）核心稅費計算"; apply_styles_to_range(ws, 'A17', {'font': Styles.font_section_header, 'fill': Styles.fill_header_grey, 'alignment': Styles.align_center})
    for i, h in enumerate(["項目", "賣方持有期間", "買方持有期間", "年度總稅額", "找補金額", "", "備註"], 1): ws.cell(row=18, column=i, value=h)
    ws.merge_cells('E18:G18')
    apply_styles_to_range(ws, 'A18:H18', {'font': Styles.font_body_bold, 'fill': Styles.fill_header_grey, 'alignment': Styles.align_center})

    ws['A19']="地價稅"; ws.merge_cells('E19:G19')
    ws['B19']='=IF(AB10<>"", AB10 & IF(AA5, " 月", " 天"), "")'; ws['C19']='=IF(AB11<>"", AB11 & IF(AA5, " 月", " 天"), "")'
    ws['D19']='=AB1'; ws['E19']='=IFERROR(IF(AB8="賣方", ROUND(D19*AB11/AB3,0), ROUND(D19*AB10/AB3,0)),0)'
    ws['H19']='=IF(E19=0, "無需找補", "納稅人為【" & AB8 & "】，" & IF(AB8="賣方", "由【買方】補貼賣方", "由【賣方】補貼買方") & "。" & CHAR(10) & "【找補計算】" & CHAR(10) & "  " & TEXT(D19, "#,##0") & " * (" & IF(AB8="賣方", AB11, AB10) & " / " & AB3 & ") = " & TEXT(E19, "#,##0") & CHAR(10) & "【款項性質】" & CHAR(10) & "  " & "此為民國 " & YEAR(AA2)-1911 & " 年度地價稅找補，屬【" & IF(AA2>DATE(YEAR(AA2),8,31), "未到期數", "已到期數") & "】款項。" & CHAR(10) & "---" & CHAR(10) & "【稅務規則】" & CHAR(10) & "● 課稅年度： 民國 " & YEAR(AA2)-1911 & " 年 (1/1 ~ 12/31)" & CHAR(10) & "● 開徵期間： 民國 " & YEAR(AA2)-1911 & " 年 11 月 1 日 至 11 月 30 日" & CHAR(10) & "● 納稅基準日： 民國 " & YEAR(AA2)-1911 & " 年 8 月 31 日" & CHAR(10) & "● 本案納稅人： 【" & AB8 & "】 (以基準日之地政登記所有權人為準)")'

    ws['A20']="房屋稅"; ws.merge_cells('E20:G20')
    ws['B20']='=IF(AB12<>"", AB12 & IF(AA5, " 月", " 天"), "")'; ws['C20']='=IF(AB13<>"", AB13 & IF(AA5, " 月", " 天"), "")'
    ws['D20']='=AB2'; ws['E20']='=IFERROR(IF(AB9="賣方", ROUND(D20*AB13/AB6,0), ROUND(D20*AB12/AB6,0)),0)'
    ws['H20']='=IF(E20=0, "無需找補", "納稅人為【" & AB9 & "】，" & IF(AB9="賣方", "由【買方】補貼賣方", "由【賣方】補貼買方") & "。" & CHAR(10) & IF(C16="", "【年稅額計算】" & CHAR(10) & "  " & IF(AA17>0, "(" & TEXT(AA16,"#,##0") & "-" & TEXT(AA17,"#,##0") & ")", TEXT(AA18,"#,##0")) & " * " & TEXT(D13,"0.0%") & " = " & TEXT(D20, "#,##0") & CHAR(10), "") & "【找補計算】" & CHAR(10) & "  " & TEXT(D20, "#,##0") & " * (" & IF(AB9="賣方", AB13, AB12) & " / " & AB6 & ") = " & TEXT(E20, "#,##0") & CHAR(10) & "【款項性質】" & CHAR(10) & "  " & "此為民國 " & YEAR(AB5)-1911 & " 年度房屋稅找補，屬【" & IF(AA2>AB7, "未到期數", "已到期數") & "】款項。" & CHAR(10) & "---" & CHAR(10) & "【稅務規則】" & CHAR(10) & "● 課稅期間： 民國 " & YEAR(AB4)-1911 & "/7/1 至 " & YEAR(AB5)-1911 & "/6/30" & CHAR(10) & "● 開徵期間： 民國 " & YEAR(AB5)-1911 & " 年 5 月 1 日 至 5 月 31 日" & CHAR(10) & "● 納稅基準日： " & TEXT(AB7, "民國 e 年 m 月 d 日") & CHAR(10) & "● 本案納稅人： 【" & AB9 & "】 (以基準日之房屋所有權人為準)")'

    apply_styles_to_range(ws, 'B19:C19', {'fill': Styles.fill_period_land}); apply_styles_to_range(ws, 'B20:C20', {'fill': Styles.fill_period_house})
    apply_styles_to_range(ws, 'A19:H20', {'font': Styles.font_body, 'alignment': Styles.align_center})
    apply_styles_to_range(ws, 'H19:H20', {'alignment': Styles.align_left_top}); apply_styles_to_range(ws, 'D19:G20', {'number_format': '#,##0'})

    # Section 3
    ws.merge_cells('A21:H21'); ws['A21'] = "（三）其他費用分算 (公用事業 & 管理費)"
    apply_styles_to_range(ws, 'A21', {'font': Styles.font_section_header, 'fill': Styles.fill_header_grey, 'alignment': Styles.align_center})
    
    ws['A22'] = "管理費(月)"; ws.merge_cells('D22:E22'); ws['D22'] = "管理費已預繳至"
    ws.merge_cells('G22:H22'); ws['G22'] = '=IF(AA20<>"", TEXT(AA20, "民國 e 年 m 月 d 日"), "格式: YYYMMDD")'
    ws['A23'] = "車位費(月)"; ws.merge_cells('D23:E23'); ws['D23'] = "車位費已預繳至"
    ws.merge_cells('G23:H23'); ws['G23'] = '=IF(AA21<>"", TEXT(AA21, "民國 e 年 m 月 d 日"), "格式: YYYMMDD")'
    apply_styles_to_range(ws, 'B22:B23', {'fill': Styles.fill_yellow_input})
    apply_styles_to_range(ws, 'F22:F23', {'fill': Styles.fill_yellow_input, 'number_format': '@'})

    ws.merge_cells('A24:H24')
    ws['A24'].value = "⚠️ 租金/押金 計算請至【交屋租金分算表】分頁填寫，結果將自動彙整至下方結算區。"
    apply_styles_to_range(ws, 'A24', {'font': Styles.font_red_bold, 'alignment': Styles.align_center, 'fill': Styles.fill_result_orange})

    ws['A25'] = "水費溢繳"; ws['A26'] = "電費溢繳"; ws['A27'] = "瓦斯費溢繳"
    apply_styles_to_range(ws, 'B25:B27', {'fill': Styles.fill_yellow_input})
    apply_styles_to_range(ws, 'A22:H27', {'font': Styles.font_body, 'alignment': Styles.align_right})
    apply_styles_to_range(ws, 'G22:H23', {'alignment': Styles.align_left})

    # Section 4
    ws.merge_cells('A28:H28'); ws['A28'] = "（四）最終結算"; apply_styles_to_range(ws, 'A28', {'font': Styles.font_section_header, 'fill': Styles.fill_header_grey, 'alignment': Styles.align_center})
    ws.merge_cells('A29:C29'); ws['A29'] = "項目"; ws['D29']="買方應付給賣方"; ws['E29']="賣方應付給買方"; ws.merge_cells('F29:H29'); ws['F29']="詳細計算說明"
    apply_styles_to_range(ws, 'A29:H29', {'font': Styles.font_body_bold, 'fill': Styles.fill_header_grey, 'alignment': Styles.align_center})

    # Helper for Management Fee detailed logic (Cross month vs Same month)
    def get_mgmt_desc_formula(fee_cell, start_date, paid_date, date_cut_adj):
        # 判斷是同月還是跨月
        cond_same_month = f"(TEXT({start_date},\"yyyymm\")=TEXT({paid_date},\"yyyymm\"))"
        
        # 同月算式字串: Fee * (Days / TotalDaysInMonth)
        days_same = f"({paid_date}-{start_date}+{date_cut_adj})"
        total_days_same = f"DAY(EOMONTH({start_date},0))"
        txt_same = f"\"同月分算：$\" & TEXT({fee_cell},\"#,##0\") & \" × (\" & {days_same} & \"天 / \" & {total_days_same} & \"天)\""
        
        # 跨月算式字串: Fee * (RestDays1/Total1) + Fee * (Days2/Total2)
        days_rest_m1 = f"(DAY(EOMONTH({start_date},0))-DAY({start_date})+{date_cut_adj})"
        total_m1 = f"DAY(EOMONTH({start_date},0))"
        days_m2 = f"DAY({paid_date})"
        total_m2 = f"DAY(EOMONTH({paid_date},0))"
        
        # 組合跨月字串
        txt_cross = f"\"跨月分段：$\" & TEXT({fee_cell},\"0\") & \"×(\" & {days_rest_m1} & \"/\" & {total_m1} & \") + $\" & TEXT({fee_cell},\"0\") & \"×(\" & {days_m2} & \"/\" & {total_m2} & \")\""
        
        # 若跨超過2個月，後面加 "..."
        is_multi_month = f"((YEAR({paid_date})-YEAR({start_date}))*12 + MONTH({paid_date})-MONTH({start_date}) > 1)"
        txt_cross_final = f"IF({is_multi_month}, {txt_cross} & \" +...\", {txt_cross})"

        return f"IF({cond_same_month}, {txt_same}, {txt_cross_final})"

    # Final Items Setup (Updated with detailed formulas)
    final_items = {
        30: ("地價稅找補", '=IF(MAX(D30,E30)=0, "無需找補", "【" & IF(AB8="賣方", "買方", "賣方") & "負擔】" & CHAR(10) & "年稅 $" & TEXT(D19,"#,##0") & " × (持有" & IF(AB8="賣方", AB11, AB10) & IF(AA5,"個月","天") & " / 全年" & AB3 & IF(AA5,"個月","天") & ")")'),
        
        31: ("房屋稅找補", '=IF(MAX(D31,E31)=0, "無需找補", "【" & IF(AB9="賣方", "買方", "賣方") & "負擔】" & CHAR(10) & "年稅 $" & TEXT(D20,"#,##0") & " × (持有" & IF(AB9="賣方", AB13, AB12) & IF(AA5,"個月","天") & " / 全年" & AB6 & IF(AA5,"個月","天") & ")")'),
        
        32: ("管理費找補", f'=IF(D32>0, {get_mgmt_desc_formula("B22", "AA2", "AA20", "AA4")}, IF(E32>0, {get_mgmt_desc_formula("B22", "AA20", "AA2", "IF(AA4,0,1)")}, "無需找補"))'),
        
        33: ("車位費找補", f'=IF(D33>0, {get_mgmt_desc_formula("B23", "AA2", "AA21", "AA4")}, IF(E33>0, {get_mgmt_desc_formula("B23", "AA21", "AA2", "IF(AA4,0,1)")}, "無需找補"))'),
        
        34: ("租金/押金找補總額", '="詳見【交屋租金分算表】。賣方應付小計(含押金轉移) vs 買方應付小計(含欠租補貼)"'),
        
        35: ("水電瓦斯溢繳", '="明細：水費$" & TEXT(B25,"#,##0") & " + 電費$" & TEXT(B26,"#,##0") & " + 瓦斯$" & TEXT(B27,"#,##0") & " = 總計$" & TEXT(D35,"#,##0")'),
        
        36: ("其他 (買方補貼賣方)", "手動輸入雙方約定之其他費用 (應付賣方)"),
        37: ("其他 (賣方補貼買方)", "手動輸入雙方約定之其他費用 (應付買方)")
    }

    for r, (label, desc) in final_items.items():
        ws.merge_cells(f'A{r}:C{r}'); ws[f'A{r}']=label; ws.merge_cells(f'F{r}:H{r}'); ws[f'F{r}']=desc

    ws.merge_cells('A38:C38'); ws['A38']="款項小計"; apply_styles_to_range(ws, 'A38', {'font': Styles.font_body_bold})
    apply_styles_to_range(ws, 'A30:C38', {'font': Styles.font_body, 'alignment': Styles.align_left})
    
    # 修正處：移除 wrap_text: True，改為直接使用 align_left_top
    apply_styles_to_range(ws, 'F30:H37', {'font': Styles.font_note, 'alignment': Styles.align_left_top})
    
    apply_styles_to_range(ws, 'D30:E38', {'number_format': '#,##0', 'alignment': Styles.align_center})
    apply_styles_to_range(ws, 'D30:D38', {'font': Styles.font_buyer_pays}); apply_styles_to_range(ws, 'E30:E38', {'font': Styles.font_seller_pays})
    
    apply_styles_to_range(ws, 'D36', {'fill': Styles.fill_yellow_input}) 
    apply_styles_to_range(ws, 'E37', {'fill': Styles.fill_yellow_input}) 

    ws.merge_cells('A39:B40'); ws['A39']="最終結算結果"; apply_styles_to_range(ws, 'A39', {'font': Styles.font_section_header, 'fill': Styles.fill_result_orange, 'alignment': Styles.align_center})
    ws.merge_cells('C39:H40'); apply_styles_to_range(ws, 'C39', {'font': Styles.font_final_result, 'fill': Styles.fill_result_orange, 'alignment': Styles.align_center})

    # Linking Logic
    ws['D30']='=IF(AB8="賣方", E19, 0)'; ws['E30']='=IF(AB8="買方", E19, 0)'
    ws['D31']='=IF(AB9="賣方", E20, 0)'; ws['E31']='=IF(AB9="買方", E20, 0)'
    ws['D32']='=AB21'; ws['E32']='=AB24'
    ws['D33']='=AB22'; ws['E33']='=AB25'
    ws['D34'] = "=" + rent_subtotals_refs[1] 
    ws['E34'] = "=" + rent_subtotals_refs[0]
    ws['D35']='=SUM(B25:B27)'; ws['E35']='=0'
    ws['E36'] = 0; ws['D37'] = 0
    ws['D38']='=SUM(D30:D37)'; ws['E38']='=SUM(E30:E37)'
    ws['C39']='=IF(D38>E38, "【買方】應支付給【賣方】 NT$ " & TEXT(D38-E38, "#,##0"), IF(E38>D38, "【賣方】應支付給【買方】 NT$ " & TEXT(E38-D38, "#,##0"), "雙方無需找補"))'

    # Footer & Borders
    ws.merge_cells('A42:H42'); ws['A42'] = "工具開發：陳定康 (Integrated Version)"; apply_styles_to_range(ws, 'A42', {'font': Styles.font_footer, 'alignment': Styles.align_right})
    for r in ['A2:H4', 'A5:H13', 'A14:C16', 'A17:H20', 'A21:H27', 'A28:H40']: apply_styles_to_range(ws, r, {'border': Styles.thin_border})
    apply_styles_to_range(ws, 'A39:H40', {'border': Styles.medium_border})
    apply_styles_to_range(ws, 'F2:H2', {'border': Styles.thin_border})

    dv_land = DataValidation(type="list", formula1='"自用住宅 (2‰),一般用地 (10‰)"'); dv_land.add('F7:F9'); ws.add_data_validation(dv_land)
    dv_house = DataValidation(type="list", formula1="=稅率表!$A$2:$A$15"); dv_house.add('C13'); ws.add_data_validation(dv_house)

# --- 產生最終檔案 ---
def create_combined_workbook(filename="交屋稅費租金整合版v3_3.xlsx"):
    wb = Workbook()
    wb.remove(wb.active)
    
    print("正在建立租金分算表...")
    rent_refs = generate_rent_sheet(wb)
    
    print("正在建立稅費主表 (含詳細算式說明)...")
    generate_tax_sheet(wb, rent_refs)
    
    print("正在建立稅率表...")
    ws_rate = wb.create_sheet("稅率表")
    rates = [("房屋使用情境選項", "對應稅率"),("自住用-全國單一自住(現值一定金額下)", 0.01),("自住用-全國3戶內", 0.012),("公益出租人/社會住宅", 0.012),
             ("非自住-出租(達租金標準)/繼承共有-4戶內", 0.015),("非自住-出租(達租金標準)/繼承共有-5~6戶", 0.02),("非自住-出租(達租金標準)/繼承共有-7戶以上", 0.024),
             ("非自住-其他住家用-2戶內", 0.032),("非自住-其他住家用-3~4戶", 0.038),("非自住-其他住家用-5~6戶", 0.042),("非自住-其他住家用-7戶以上", 0.048),
             ("營業用", 0.03),("私人醫院/診所/事務所用", 0.03),("非住家非營業用(人民團體等)", 0.02)]
    for r, row in enumerate(rates, 1):
        for c, val in enumerate(row, 1): ws_rate.cell(row=r, column=c, value=val)
    ws_rate.column_dimensions['A'].width = 50
    
    wb.move_sheet(wb['專業稅費分算計算機'], offset=-1)
    
    wb.save(filename)
    print(f"✅ 成功產出整合版檔案：{filename}")

if __name__ == '__main__':
    create_combined_workbook()


正在建立租金分算表...
正在建立稅費主表 (含詳細算式說明)...
正在建立稅率表...
✅ 成功產出整合版檔案：交屋稅費租金整合版v3_3.xlsx
